### U-Net Implementation of Monocular Depth Estimation

In [1]:
import torch
import torch.nn as nn
from torch.nn import init
import functools

if torch.cuda.is_available():
    # Set Device
    torch.cuda.set_device(0)
    print("Cuda enabled on device: {}".format(torch.cuda.current_device()))


Cuda enabled on device: 0


In [19]:
#%%writefile depthnet/model.py
import torch
import torch.nn as nn
from torch.nn import init
from collections import OrderedDict

def create_sublayer_dict(input_nc, output_nc, layer_index, nconvs, norm_layer, **conv_kwargs):
    sublayer=OrderedDict()
    j = 0
    sublayer.update({"conv{}_{}".format(layer_index, j): nn.Conv2d(input_nc, output_nc, **conv_kwargs)})
    j += 1
    sublayer.update({"relu{}_{}".format(layer_index, j): nn.ReLU(True)})
    for i in range(nconvs-1):
        j += 1
        sublayer.update({"conv{}_{}".format(layer_index, j): nn.Conv2d(output_nc, output_nc, **conv_kwargs)})
        j += 1
        sublayer.update({"relu{}_{}".format(layer_index, j): nn.ReLU(True)})
    if norm_layer:
        j += 1
        sublayer.update({"norm{}_{}".format(layer_index, j): norm_layer(output_nc)})
    return sublayer

class DepthNet(nn.Module):
    def __init__(self, input_nc=3, output_nc=1, norm_layer=nn.BatchNorm2d):
        super(DepthNet, self).__init__()
        self.input_nc = input_nc
        self.output_nc = output_nc
        use_bias = True

        # Conv1
        model1 = create_sublayer_dict(input_nc, 32, 1, 2, norm_layer, 
                                      kernel_size=3, stride=1, padding=1, bias=use_bias)
        # add a subsampling operation (in self.forward())

        # Conv2
        model2 = create_sublayer_dict(32, 64, 2, 2, norm_layer,
                                      kernel_size=3, stride=1, padding=1, bias=use_bias)
        # add a subsampling layer operation (in self.forward())

        # Conv3
        model3 = create_sublayer_dict(64, 128, 3, 3, norm_layer,
                                      kernel_size=3, stride=1, padding=1, bias=use_bias)
        # add a subsampling layer operation

        # Conv4
        model4 = create_sublayer_dict(128, 256, 4, 3, norm_layer,
                                      kernel_size=3, stride=1, padding=1, bias=use_bias)

        # Conv5
        model5 = create_sublayer_dict(256, 256, 5, 3, norm_layer,
                                      kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias)

        # Conv6
        model6 = create_sublayer_dict(256, 256, 6, 3, norm_layer,
                                      kernel_size=3, dilation=2, stride=1, padding=2, bias=use_bias)

        # Conv7
        model7 = create_sublayer_dict(256, 256, 7, 3, norm_layer,
                                      kernel_size=3, stride=1, padding=1, bias=use_bias)

        # Conv7
        model8up=OrderedDict([("convt8_up", 
                               nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=use_bias))]
                            )
        model3short8=OrderedDict([("conv3short8",
                                   nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=use_bias))]
                                )

        model8=OrderedDict([("relu8_pre", nn.ReLU(True))])

        model8.update(create_sublayer_dict(128, 128, 8, 2, norm_layer,
                                      kernel_size=3, stride=1, padding=1, bias=use_bias))

        # Conv9
        model9up=OrderedDict([("convt9_up", 
                               nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=use_bias))]
                            )
        model2short9=OrderedDict([("conv2short9",
                                   nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=use_bias))]
                                )
        # add the two feature maps above        

        model9=OrderedDict([("relu9_pre", nn.ReLU(True))])

        model9.update(create_sublayer_dict(64, 64, 9, 1, norm_layer,
                                           kernel_size=3, stride=1, padding=1, bias=use_bias))

        # Conv10
        model10up=OrderedDict([("conv10_up", 
                               nn.ConvTranspose2d(64, 64, kernel_size=4, stride=2, padding=1, bias=use_bias))]
                             )
        model1short10=OrderedDict([("conv1short10", 
                                    nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1, bias=use_bias))]
                                 )
        # add the two feature maps above

        model10=OrderedDict([("relu10_pre", nn.ReLU(True))])
        model10.update({"conv10_0": nn.Conv2d(64, 64, kernel_size=3, dilation=1, stride=1, padding=1, bias=use_bias)})
        model10.update({"leakyrelu10_1": nn.LeakyReLU(negative_slope=.2)})

        # Depth Map Regression Output
        model_out=OrderedDict([("conv_out",
                                nn.Conv2d(64, 1, kernel_size=1, padding=0, dilation=1, stride=1, bias=use_bias))]
                             )
        model_out.update({"relu_out": nn.ReLU(True)}) # Depth should be in [0, +inf)

        self.model1 = nn.Sequential(model1)
        self.model2 = nn.Sequential(model2)
        self.model3 = nn.Sequential(model3)
        self.model4 = nn.Sequential(model4)
        self.model5 = nn.Sequential(model5)
        self.model6 = nn.Sequential(model6)
        self.model7 = nn.Sequential(model7)
        self.model8up = nn.Sequential(model8up)
        self.model8 = nn.Sequential(model8)
        self.model9up = nn.Sequential(model9up)
        self.model9 = nn.Sequential(model9)
        self.model10up = nn.Sequential(model10up)
        self.model10 = nn.Sequential(model10)
        self.model3short8 = nn.Sequential(model3short8)
        self.model2short9 = nn.Sequential(model2short9)
        self.model1short10 = nn.Sequential(model1short10)
        self.model_out = nn.Sequential(model_out)

    def forward(self, input_A):
        conv1_2 = self.model1(input_A)
        conv2_2 = self.model2(conv1_2[:,:,::2,::2]) # downsample
        conv3_3 = self.model3(conv2_2[:,:,::2,::2]) # downsample
        conv4_3 = self.model4(conv3_3[:,:,::2,::2]) # downsample
        conv5_3 = self.model5(conv4_3)
        conv6_3 = self.model6(conv5_3)
        conv7_3 = self.model7(conv6_3)
        conv8_up = self.model8up(conv7_3) + self.model3short8(conv3_3) # Shortcut
        conv8_3 = self.model8(conv8_up)
        conv9_up = self.model9up(conv8_3) + self.model2short9(conv2_2) # Shortcut
        conv9_3 = self.model9(conv9_up)
        conv10_up = self.model10up(conv9_3) + self.model1short10(conv1_2) # Shortcut
        conv10_2 = self.model10(conv10_up)
        out_reg = self.model_out(conv10_2)

        return out_reg

##################
# Loss functions #
##################

def berhu(prediction, target):
    diff = prediction - target
    threshold = 0.2*torch.max(torch.abs(prediction - target))
    c = threshold.detach()
    l2_part = torch.sum((diff**2 + c**2))/(2*c)
    l1_part = torch.sum(torch.abs(diff))
    return l1_part+l2_part


Overwriting depthnet/model.py


In [18]:
#%%writefile depthnet/data.py
###########
# Dataset #
###########
from PIL import Image
import torch
from torch.utils.data import Dataset
import csv, numpy as np
import os
from collections import defaultdict
from torchvision import transforms

class DepthDataset(Dataset):
    """Class for reading and storing image and depth data together.
    """
    def __init__(self, splitfile, dataDir, transform=None):
        """
        Parameters
        ----------
        images : list of (string, string)
            list of (depth_map_path, rgb_path) filepaths to depth maps and their rgb images.
        load_depth_map : function
            the function for loading this particular kind of depth_map
        load_rgb : function
            the function for loading this particular kind of image.
        """
        super(DepthDataset, self).__init__()
        self.dataDir = dataDir
        self.transform = transform
        self.data = []
        with open(splitfile, "r") as f:
            for line in f.readlines():
                self.data.append(line.strip().split(","))
#         print(self.data)
    
    def get_global_stats(self, outFile=None, writeFile=False):
        """Calculate mean and variance of each rgb channel.
        
        Optionally caches the result of this calculation in outfile so it doesn't need to be done each
        time the dataset is loaded.
        """
        S = np.zeros(3)
        S_sq = np.zeros(3)
        npixels = 0.
        for depthFile, rgbFile in self.data:
            rgbImg = Image.open(os.path.join(self.dataDir, rgbFile))
            rgbImg = np.asarray(rgbImg, dtype=np.uint16)
#             print(rgbImg[0:10, 0:10, :])
            
            npixels += rgbImg.shape[0]*rgbImg.shape[1]
            for channel in range(rgbImg.shape[2]):
                S[channel] += np.sum(rgbImg[:,:,channel])
                S_sq[channel] += np.sum((rgbImg[:,:,channel])**2)
        mean = S/npixels
        var = S_sq/npixels - mean**2
        
        # Load full dataset (memory-intensive)
#         full = []
#         for depthFile, rgbFile in self.data:
#             rgbImg = Image.open(os.path.join(self.dataDir, rgbFile))
#             rgbImg = np.asarray(rgbImg, dtype=np.uint16)
#             full.append(rgbImg)
            
#         a = np.array(full)
#         mean_true = np.mean(a, axis=(0, 1, 2))
#         var_true = np.var(a, axis=(0, 1, 2))
#         print("actual mean and variance: {} {}".format(mean_true, var_true))
#         print(a.shape)
        return mean, var
                
        
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        depthFile, rgbFile = self.data[idx]
        depthImg = Image.open(os.path.join(self.dataDir, depthFile))
        rgbImg = Image.open(os.path.join(self.dataDir, rgbFile))
        sample = {"depth": depthImg, "rgb": rgbImg}
        if self.transform:
            sample = self.transform(sample)
        return sample

#############
# Load data #
#############

def load_data(trainFile, trainDir, devFile, devDir, trainTransform=None, valTransform=None):
    """Generates training and validation datasets from
    text files and directories. Sets up datasets with transforms."""
    train = DepthDataset(trainFile, trainDir)
    mean, var = train.get_global_stats()
    train.transform = trainTransform
    print("Loaded training dataset from {} with size {}.".format(train_txt, len(train)))

    val = DepthDataset(valFile, valDir, transform = valTransform)
    print("Loaded val dataset from {} with size {}.".format(dev_txt, len(dev)))
    return train, val
    
##############
# Transforms #
##############
# for data augmentation
class RandomCrop(object):
    """Crop randomly the image in a sample.

    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size

    def __call__(self, sample):
        depth, rgb = sample['depth'], sample['rgb']

        h, w = depth.shape[:2]
        new_h, new_w = self.output_size

        top = np.random.randint(0, h - new_h)
        left = np.random.randint(0, w - new_w)

        depth = depth[top: top + new_h,
                      left: left + new_w]
        
        rgb = rgb[top: top + new_h,
                  left: left + new_w]

        return {'depth': depth, 'rgb': rgb}

class CenterCrop(object):
    """Center crop the image
    
    Args:
        output_size (tuple or int): Desired output size. If int, square crop
            is made.
    """
    
    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        if isinstance(output_size, int):
            self.output_size = (output_size, output_size)
        else:
            assert len(output_size) == 2
            self.output_size = output_size
            
    def __call__(self, sample):
        depth, rgb = sample['depth'], sample['rgb']
        h, w = depth.shape
        new_h, new_w = self.output_size

        top = h//2 - new_h//2
        bottom = h//2 + new_h//2 + (1 if new_h % 2 else 0)
        left = w//2 - new_w//2
        right = w//2 + new_w//2 + (1 if new_w % 2 else 0)
        
        return {"depth": depth[top:bottom, left:right],
                "rgb": rgb[top:bottom, left:right, :]}
    
class Crop_8(object):
    """Crop to a size where both dimensions are divisible by 8"""
    
    def __call__(self, sample):
        depth, rgb = sample['depth'], sample['rgb']
        new_h, new_w = (depth.shape[0]//8)*8, (depth.shape[1]//8)*8
        return {"depth": depth[:new_h, :new_w],
                "rgb": rgb[:new_h, :new_w, :]}
        
class Crop_small(object):
    def __call__(self, sample):
        depth, rgb = sample['depth'], sample['rgb']
        h, w = depth.shape[:2]
        x = 16
        return {"depth": depth[h//2-x:h//2+x, w//2-x:w//2+x],
                "rgb": rgb[h//2-x:h//2+x, w//2-x:w//2+x, :]}

class ToFloat(object):
    """Also parses the depth info for sunrgbd."""
    def __call__(self, sample):
        depth = sample['depth']
        x = np.asarray(depth, dtype=np.uint16)
        y = (x >> 3) | (x << 16-3)
        z = y.astype(np.float32)/1000
        z[z>8.] = 8. # Clip to maximum depth of 8m.
        return {"depth": z,
                "rgb": np.asarray(sample['rgb']).astype(np.float32)}
    
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        depth, rgb = sample['depth'], sample['rgb']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
#         depth = depth.transpose((2, 0, 1))
        rgb = rgb.transpose((2, 0, 1))            
        output = {}
        if 'hist' in sample:
            output['hist'] = torch.from_numpy(sample['hist']).unsqueeze(-1).unsqueeze(-1)

#         print(output)
        output.update({'depth': torch.from_numpy(depth).unsqueeze(0),
                'rgb': torch.from_numpy(rgb)})
        return output
    
class AddDepthHist(object):
    """Takes a depth map and computes a histogram of depths as well"""
    def __init__(self, **kwargs):
        """
        kwargs - passthrough to np.histogram
        """
        self.hist_kwargs = kwargs
        
    def __call__(self, sample):
        depth = sample["depth"]
        hist, _ = np.histogram(depth, **self.hist_kwargs)
#         print(hist)
#         print(sample["depth"])
        return {"depth": sample["depth"],
                "rgb": sample["rgb"],
                "hist": hist}

class NormalizeRGB(object):
    def __init__(self, mean, var):
        """
        mean - np.array of size 3 - the means of the three color channels over the whole (training) dataset
        var - np.array of size 3 - the variances of the three color channels over the whole (training) dataset
        """
        self.mean = mean
        self.var = var
    def __call__(self, sample):
        sample["rgb"] -= self.mean
        sample["rgb"] /= np.sqrt(self.var)
#         print(sample["rgb"][0:10, 0:10, 0])
        return sample



Writing depthnet/data.py


In [ ]:
#%%writefile depthnet/utils.py
import torch
from torch.utils.data import Dataset, DataLoader
import torch.cuda as cuda
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms
import os.path
# Helper functions

#################
# Checkpointing #
#################
def save_checkpoint(state, is_best, filename='/output/checkpoint.pth.tar', always_save=False):
    """Save checkpoint if a new best is achieved"""
    if is_best or always_save:
        print ("=> Saving checkpoint to: {}".format(filename))
        torch.save(state, filename)  # save checkpoint
    else:
        print ("=> Validation Accuracy did not improve")

##############
# Validation #
##############
def validate(loss, model, val_loader):
    """Computes the validation error of the model on the validation set.
    val_loader should be a DataLoader.
    
    Returns an ordinary number (i.e. not a tensor)
    
    """
    
    it = None
    losses = []
    for it, data in enumerate(val_loader):
        depth = data["depth"].float()
        rgb = data["rgb"].float()
        if torch.cuda.is_available():
            depth = depth.cuda()
            rgb = rgb.cuda()
        if "hist" in data:
#             print(data)
            hist = data["hist"].float()
            if torch.cuda.is_available():
                hist = hist.cuda()
#             print(hist)
            output = model(rgb, hist)
        else:
            output = model(rgb)
        losses.append(loss(output, depth).item())
    nbatches = it+1
    return sum(losses)/nbatches

##################
# Viewing Images #
##################
def save_images(*batches, outputDir, filename):
    """
    Given a list of tensors of size (B, C, H, W) (batch, channels, height, width) torch.Tensor
    Saves each entry of the batch as an rgb or grayscale image, depending on how many channels
    the image has.
    """
    I = None
    trans = transforms.ToPILImage()
    for batchnum, batch in enumerate(batches):
        if batch.shape[1] == 3:
            pass
        elif batch.shape[1] == 1:
            batch /= torch.max(batch) # normalize to lie in [0, 1]
        else:
            raise ValueError("Unsupported number of channels: {}".format(batch.shape[1]))
        batch = batch.type(torch.float32)
        for img in range(batch.shape[0]):            
            I = trans(batch[img,:,:,:].cpu().detach())
            I.save(os.path.join(outputDir, filename + "_{}_{}.png".format(batchnum, img)))


############
# Plotting #
############
def save_train_val_loss_plots(trainlosses, vallosses, epoch):
    # Train loss
    fig = plt.figure()
    plt.plot(trainlosses)
    plt.title("Train loss")
    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.savefig("trainloss_epoch{}.png".format(epoch))
    # Train loss
    fig = plt.figure()
    plt.plot(trainlosses)
    plt.title("Val loss")
    plt.xlabel("epoch")
    plt.ylabel("Loss")
    plt.savefig("Val loss{}.png".format(epoch))

In [ ]:
# Load training data
def load_data(train_file, train_dir, dev_file, dev_dir)
    train_txt = "data/sunrgbd_nyu/train.txt"
    trainDir = "data/sunrgbd_nyu"
    train = DepthDataset(train_txt, trainDir)
    mean, var = train.get_global_stats()
    train.transform = transforms.Compose([ToFloat(),
    #                                       RandomCrop((400, 320)),
                                          CenterCrop((320, 400)),
                                          NormalizeRGB(mean, var),
                                          ToTensor()
                                          ])


    #                      transform=transforms.Compose([ToFloat(), Crop_8(), ToFloat(), ToTensor()])
    #                      transform=transforms.Compose([ToFloat(), Crop_small(), ToFloat(), ToTensor()])
    # print(mean, var)

    print("Loaded training dataset from {} with size {}.".format(train_txt, len(train)))

    dev_txt = "data/sunrgbd_nyu/dev.txt"
    devDir = "data/sunrgbd_nyu"
    dev = DepthDataset(dev_txt, devDir, 
                         transform = transforms.Compose([ToFloat(),
                                                         CenterCrop((320, 400)), 
                                                         NormalizeRGB(mean, var),
                                                         ToTensor(),])
    #                      transform=transforms.Compose([ToFloat(), Crop_8(), ToTensor()])
    #                      transform=transforms.Compose([ToFloat(), Crop_small(), ToTensor()])
                        )
    print("Loaded dev dataset from {} with size {}.".format(dev_txt, len(dev)))
    return train, val


Loaded training dataset from data/sunrgbd_nyu/train.txt with size 1159.
Loaded dev dataset from data/sunrgbd_nyu/dev.txt with size 145.


In [23]:
#%%writefile depthnet/train_utils.py
# Set up training.
import torch.optim as optim
from tensorboardX import SummaryWriter
import torchvision.utils as vutils
from depthnet.model import DepthNet

def setup_training(opt):
    # Build model and loss
    # Hyperparameters

    model = DepthNet(opt.input_nc, opt.output_nc)
    if torch.cuda.is_available():
        model.cuda()

    # Checkpointing
    if checkpointfile is not None:
        if torch.cuda.is_available():
            checkpoint = torch.load(checkpointfile)
        else:
            # Load GPU model on CPU
            checkpoint = torch.load(checkpointfile,
                                    map_location=lambda storage,
                                    loc: storage)
        start_epoch = checkpoint['epoch'] + 1
        best_loss = checkpoint['best_loss']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        optimizer.load_state_dict(checkpoint['optim_state_dict'])
        trainlosses = checkpoint['trainlosses']
        for i, trainloss in enumerate(trainlosses): # For tensorboardx
            writer.add_scalar("data/trainloss", trainloss, i)
        vallosses = checkpoint['vallosses']
        for i, valloss in enumerate(vallosses): # For tensorboardx
            writer.add_scalar("data/valloss", valloss, i)
        global_it = len(trainlosses)
        print("=> loaded checkpoint '{}' (trained for {} epoch(s)).".format(checkpointfile, start_epoch))
    else:
        start_epoch = 0
        global_it = 0 # Track global iterations
        best_loss = torch.FloatTensor([float('inf')])
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
        trainlosses = []
        vallosses = []
        # Initialize weights:
        for name, param in model.named_parameters():
            if "conv" in name and "weight" in name:
    #             print(name)
                nn.init.xavier_normal_(param)
            if "norm" in name and "weight" in name:
    #             print(name)
                nn.init.constant_(param, 1)
            elif "bias" in name:
                nn.init.constant_(param, 0)

    # Scheduler
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[], gamma=0.1)
    scheduler.last_epoch = start_epoch - 1

    # Print summary of setup:
    print("loaded checkpointfile: {}".format(checkpointfile))
    print("start_epoch: {}".format(start_epoch))
    print("global_it: {}".format(global_it))
    print("optimizer: {}".format(optimizer))
    print("batch_size: {}".format(batch_size))
    print("num_epochs: {}".format(num_epochs))
    print("learning rate (initial): {}".format(learning_rate))
    print("scheduler: {}".format(scheduler.state_dict()))
    
    out = {"model": model,
           "start_epoch": start_epoch,
           "num_epochs": num_epochs,
           "global_it": global_it,
           "scheduler": scheduler
          }
    return out

####################
# Run the training #
####################
def train(model, loss, start_epoch, num_epochs, global_it, scheduler, trainLoader, valLoader=None, test_run=False, writer=None):
    for epoch in range(start_epoch, start_epoch + num_epochs):
        print("epoch: {}".format(epoch))
        data = None
        output = None
        for it, data in enumerate(trainLoader):
            depth = data["depth"].float()
            rgb = data["rgb"].float()

            if torch.cuda.is_available():
                depth = depth.cuda()
                rgb = rgb.cuda()
            # New batch
            scheduler.optimizer.zero_grad()
            output = model(rgb)
            trainloss = loss(output, depth)
            trainloss.backward()
            scheduler.optimizer.step()
            global_it += 1

            if not (it % 10):
                print("\titeration: {}\ttrain loss: {}".format(it, trainloss.item()))
            trainlosses.append(trainloss.item())
            writer.add_scalar("data/trainloss", trainloss.item(), global_it)

            if test_run: # Stop after 5 batches
                if not ((it + 1) % 5):
                    break
        # Checkpointing
        if valLoader is not None:
            valloss = validate(loss, model, valLoader)
            print("End epoch {}\tval loss: {}".format(epoch, valloss))
            vallosses.append(valloss)
            writer.add_scalar("data/valloss", valloss, epoch)

        # Save the last batch output of every epoch
        rgb_input = vutils.make_grid(data["rgb"], nrow=batch_size, normalize=True, scale_each=True)
        writer.add_image('image/rgb_input', rgb_input, epoch)

        depth_truth = vutils.make_grid(data["depth"], nrow=batch_size, normalize=True, scale_each=True)
        writer.add_image('image/depth_truth', depth_truth, epoch)

        depth_output = vutils.make_grid(output, nrow=batch_size, normalize=True, scale_each=True)
        writer.add_image('image/depth_output', depth_output, epoch)

        for name, param in model.named_parameters():
            writer.add_histogram(name, param.clone().cpu().data.numpy(), global_it)
    #     save_images(data["rgb"], data["depth"], output, outputDir="images", filename="epoch_{}".format(epoch))

        is_best = bool(trainloss.data.cpu().numpy() < best_loss.numpy())
        # Get greater Tensor to keep track best acc
        best_loss = torch.FloatTensor(min(trainloss.data.cpu().numpy(), best_loss.numpy()))
        # Save checkpoint
        save_checkpoint({
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'best_loss': best_loss,
            'optim_state_dict': optimizer.state_dict(),
            'trainlosses': trainlosses,
            'vallosses': vallosses
        }, is_best, filename="checkpoints/checkpoint_epoch_{}.pth.tar".format(epoch), always_save=True)


Overwriting depthnet/train_utils.py


In [ ]:
####################
# Run the training #
####################
def train(model, scheduler, start_epoch, num_epochs, writer, trainLoader, valLoader=None, test_run=False):
    for epoch in range(start_epoch, start_epoch + num_epochs):
        print("epoch: {}".format(epoch))
        data = None
        output = None
        for it, data in enumerate(trainLoader):
            depth = data["depth"].float()
            rgb = data["rgb"].float()

            if torch.cuda.is_available():
                depth = depth.cuda()
                rgb = rgb.cuda()
            # New batch
            scheduler.optimizer.zero_grad()
            output = model(rgb)
            trainloss = loss(output, depth)
            trainloss.backward()
            scheduler.optimizer.step()
            global_it += 1

            if not (it % 10):
                print("\titeration: {}\ttrain loss: {}".format(it, trainloss.item()))
            trainlosses.append(trainloss.item())
            writer.add_scalar("data/trainloss", trainloss.item(), global_it)

            if test_run: # Stop after 5 batches
                if not ((it + 1) % 5):
                    break
        # Checkpointing
        if valLoader is not None:
            valloss = validate(loss, model, valLoader)
            print("End epoch {}\tval loss: {}".format(epoch, valloss))
            vallosses.append(valloss)
            writer.add_scalar("data/valloss", valloss, epoch)

        # Save the last batch output of every epoch
        rgb_input = vutils.make_grid(data["rgb"], nrow=batch_size, normalize=True, scale_each=True)
        writer.add_image('image/rgb_input', rgb_input, epoch)

        depth_truth = vutils.make_grid(data["depth"], nrow=batch_size, normalize=True, scale_each=True)
        writer.add_image('image/depth_truth', depth_truth, epoch)

        depth_output = vutils.make_grid(output, nrow=batch_size, normalize=True, scale_each=True)
        writer.add_image('image/depth_output', depth_output, epoch)

        for name, param in model.named_parameters():
            writer.add_histogram(name, param.clone().cpu().data.numpy(), global_it)
    #     save_images(data["rgb"], data["depth"], output, outputDir="images", filename="epoch_{}".format(epoch))

        is_best = bool(trainloss.data.cpu().numpy() < best_loss.numpy())
        # Get greater Tensor to keep track best acc
        best_loss = torch.FloatTensor(min(trainloss.data.cpu().numpy(), best_loss.numpy()))
        # Save checkpoint
        save_checkpoint({
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'best_loss': best_loss,
            'optim_state_dict': optimizer.state_dict(),
            'trainlosses': trainlosses,
            'vallosses': vallosses
        }, is_best, filename="checkpoints/checkpoint_epoch_{}.pth.tar".format(epoch), always_save=True)

# Close tensorboardX    
# writer.export_scalars_to_json("./all_scalars.json") # for other processing
writer.close()

epoch: 50
	iteration: 0	train loss: 0.46216171979904175
	iteration: 10	train loss: 0.2399614304304123
	iteration: 20	train loss: 0.5442972779273987
	iteration: 30	train loss: 0.288849800825119
	iteration: 40	train loss: 0.4346057176589966
	iteration: 50	train loss: 0.30571889877319336
	iteration: 60	train loss: 0.36826014518737793
	iteration: 70	train loss: 0.3332650363445282
	iteration: 80	train loss: 0.2840162515640259
	iteration: 90	train loss: 0.4139283299446106
	iteration: 100	train loss: 0.32223719358444214
	iteration: 110	train loss: 0.5264000296592712
End epoch 50	val loss: 1.5325142062943558
=> Saving checkpoint to: checkpoints/checkpoint_epoch_50.pth.tar
epoch: 51
	iteration: 0	train loss: 0.4966995120048523
	iteration: 10	train loss: 0.4325902760028839
	iteration: 20	train loss: 0.3255394399166107
	iteration: 30	train loss: 0.8746808767318726
	iteration: 40	train loss: 0.554697573184967
	iteration: 50	train loss: 0.40921124815940857


In [13]:
#%%writefile depthnet/options.py
from argparse import ArgumentParser

opt = ArgumentParser(description="Load command line options for depthnet.")
# Learning rates:
opt.add_argument("--lr", help="The initial learning rate of the model.",
                 type=float, default=1e-5)
opt.add_argument("--weight-decay", help="The strength of the L2 regularization on the weights.",
                 type=float, default=1e-8)

# Training hyperparameters:
# For reference: scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30], gamma=0.1)
opt.add_argument("--num-epochs", help="The number of epochs to train the model.",
                 type=int, default=50)
opt.add_argument("--milestones", help="Learning rate milestone epochs.", nargs="*",
                 type=int, default=[25])
opt.add_argument("--gamma", help="Gamma of multistepLR lr decay.",
                 type=float, default=0.1)

# Data Loading
opt.add_argument("trainFile", help="The location of the text file with the (depth, rgb) pairs.",
                 type=str)
opt.add_argument("trainDir", help="The folder containing the rgb-d training images.",
                 type=str)
opt.add_argument("--valFile", 
                 help="The location of the text file with the (depth, rgb) pairs for the validation dataset.",
                 type=str)
opt.add_argument("--valDir",
                 help="The location of the text file with the (depth, rgb) pairs for the validation dataset.",
                 type=str)
opt.add_argument("--batch-size", help="The batch size for training.",
                 type=int, default=10)
opt.add_argument("--val-batch-size", help="The batch size of the validation set.",
                 type=int, default=10)



Writing depthnet/options.py


In [17]:
# %%writefile depthnet/train.py
# Load training options from file
from depthnet.options import opt
# Tensorboardx
writer = SummaryWriter()
# data_trainloss = "data/trainloss"
# data_valloss = "data/valloss"
# Image = "Image"

# Load data
train, val = load_data(trainFile = opt.trainFile,
                       trainDir = opt.trainDir,
                       valFile = opt.valFile,
                       valDir = opt.valDir)

trainLoader = DataLoader(train, batch_size=opt.batch_size, shuffle=True, num_workers=4, pin_memory=True)
valLoader = DataLoader(val, batch_size=opt.val_batch_size, shuffle=True, num_workers=4, pin_memory=True)

# Load model, loss, and scheduler
loss = None
if opt.loss == "berhu":
    from depthnet.model import berhu
    loss = berhu
elif opt.loss == "l2":
    loss = MSELoss()
    if torch.cuda.is_available():
        loss.cuda()
elif opt.loss == "l1":
    loss = L1Loss()
    if torch.cuda.is_available():
        loss.cuda()

# Run Training



Writing depthnet/train.py


In [ ]:
# Visualize training loss

trainlosses = checkpoint['vallosses']
plt.semilogy(trainlosses)

In [ ]:
%matplotlib inline
from matplotlib.pyplot import imshow
print(type(rgb))
imshow(rgb)


In [ ]:
# Training
# Load data
a = torch.ones(1, requires_grad=True)
b = torch.ones(1, requires_grad=True)

In [ ]:
c = a + b
c.register_hook(print)
s = a**2 + b**2
s.register_hook(print)

In [ ]:
# e = torch.sum(c)
e = c.detach()
f = torch.sum(s)
g = f/e
g.backward()
# s.backward()
# f.backward()

In [ ]:
a.grad